Script Name: Program Data Update Script
Purpose: This script is used to automatically refresh the map with new program sites added to the Master Site Spreadsheet. Additionally, the script will also implement any edits done to old program sites. Any editing of the Master Site Spreadsheet or this code may cause it to fail.

Features:
- Delete the current program site table
- Import program sites from the Master Site Spreadsheet
- Plot the new program sites on the map

Instructions:
- To run this script, click on 'Kernel', then click on 'Restart & Run All. It may take a while for the script to run completely.
- If an API request error appears, wait for your limit to reset.

Author: Jayden Chan (jaydenc@usc.edu, jaydenchan26@gmail.com)

Date: 07-24-27

In [ ]:
# Locally installs a library necessary to access the Google Sheets API

!pip install gspread oauth2client

In [ ]:
# Imports required modules

from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json

In [ ]:
# Google Sheets API

# Service key - !! Keep this info private !!
SERVICE_ACCOUNT_JSON = """
{
  SERVICE_KEY_HERE
}

"""
service_account_info = json.loads(SERVICE_ACCOUNT_JSON)

SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_dict(service_account_info, SCOPES)
user = gspread.authorize(credentials)

In [ ]:
# Access Spreadsheet

spreadsheetID = 'SPREADSHEET_ID_HERE' # ID of the Master Site Spreadsheet
                                                               # Can be found in the URL
spreadsheet = user.open_by_key(spreadsheetID)
worksheet = spreadsheet.sheet1                                 # Assumes Master Sheet is the first sheet

data = worksheet.get_all_records()

In [ ]:
# ArcGIS Online

gis = GIS("home") # Use current ArcGIS Online session

feature_layer_item = gis.content.get("CONTENT_ID_HERE") # ID of the program site feature layer
                                                                         # Can be found in the URL
feature_layer = feature_layer_item.layers[0]

In [ ]:
# Feature Mapping

# Format: "<Spreadsheet Name>": "<ArcGIS Name>"
# ArcGIS Name can be found in Fields inside the curly brackets {}
field_mapping = {
    "County": "County",
    "Business Name": "Business_Name",
    "Phone": "Phone",
    "Address": "Address",
    "City": "City",
    "State": "State",
    "Latitude": "Latitude",
    "Longitude": "Longitude",
    "Zip": "Zip",
    "Full Address": "Full_Address",
    "License Type": "License_Type",
    "Licensed Capacity": "Licensed_Capacity",
    "21st CCLC": "F21st_CCLC",
    "Childcare Subsidy/CCDBG": "Childcare_Subsidy_CCDBG",
    "Local Municipal Funding": "Local_Municipal_Funding",
    "Private Foundations": "Private_Foundations",
    "Tuition/Fees": "Tuition_Fees",
    "WIOA": "WIOA",
    "CACFP": "CACFP",
    "Other": "Other"
}

In [ ]:
# Updating Map

features = []
for record in data:
    try:
        latitude = float(record["Latitude"])
        longitude = float(record["Longitude"])
    except ValueError:
        continue

    geometry = {
        "x": longitude,
        "y": latitude,
        "spatialReference": {"wkid": 4326}  # WGS 84
    }

    attributes = {field_mapping[key]: value for key, value in record.items() if key in field_mapping}
    
    feature = {
        "attributes": attributes,
        "geometry": geometry
    }
    features.append(feature)
    
feature_layer.delete_features(where="1=1") # Clears old data
feature_layer.edit_features(adds=features) # Adds new data

print("Script ran successfully.")